In [2]:
import numpy as np
import math
import pandas as pd

def volume_values(a,b,c,alpha, beta, gamma):
    G = np.linalg.det(np.array([[a**2,a*b*np.cos(gamma),a*c*np.cos(beta)],[a*b*np.cos(gamma),b**2,b*c*np.cos(alpha)],[a*c*np.cos(beta),b*c*np.cos(alpha),c**2]]))
    return np.sqrt(G)

def volume_vectors(a1,a2,a3):
    b=[]
    for i in np.cross(a2,a3): b.append(i)
    return abs(np.dot(a1, b))


def transform_coordinates(matrix, points):    
    transformed_points = []
    for j in range(len(points)):
        point = []
        # Apply the transformation
        for i in range(3):
            a=[]
            t=0
            for element in matrix[i]:
                a.append(element*points[j][t])
                t=t+1
            a = sum(a)
            point.append(a)
        transformed_points.append(point)
    return transformed_points

def params_and_units(vector_1, vector_2, vector_3):
    conv = 1.8897259886

    # Lattice parameters from lattice vectors
    a_vec = vector_1
    b_vec = vector_2
    c_vec = vector_3
    a = np.sqrt(a_vec[0]**2 + a_vec[1]**2 + a_vec[2]**2)
    b = np.sqrt(b_vec[0]**2 + b_vec[1]**2 + b_vec[2]**2)
    c = np.sqrt(c_vec[0]**2 + c_vec[1]**2 + c_vec[2]**2)
    alpha = math.acos(np.dot(b_vec, c_vec)/(b*c))
    beta = math.acos(np.dot(a_vec, c_vec)/(a*c))
    gamma = math.acos(np.dot(a_vec, b_vec)/(a*b))
    return a*conv, b*conv, c*conv, alpha*180/np.pi, beta*180/np.pi, gamma*180/np.pi

def fractional_coordinates(vector_1, vector_2, vector_3, df):

    # Convert the dataframe to a tuple
    coords = []

    for i in range(len(df)):
        coords.append((df['x'][i], df['y'][i], df['z'][i]))

    # Conversion factors for armstrong to bohr, and for the transformation matrix
    conv = 1.8897259886
    new_lattice_param_A_B = params_and_units(vector_1, vector_2, vector_3)[0]
    new_lattice_param_C = params_and_units(vector_1, vector_2, vector_3)[2]

    # Transform coordinates to 
    coords_bohr = []
    for coord in coords:
        a = []
        for i in coord:
            a.append(i*conv)
        coords_bohr.append(a)
    df_cartesian_coords_bohr = pd.DataFrame(coords_bohr)

    #Transformation matrix in bohr
    transformation_matrix_bohr = np.array([
        [1/(new_lattice_param_A_B),    1/(new_lattice_param_A_B*np.sqrt(3)),   0],  # Shift x by 2
        [0,         1/(new_lattice_param_A_B*np.sqrt(3)/2),     0],  # Shift y by 3
        [0,         0,                       1/(new_lattice_param_C)]  # Shift z by 4
    ])

    # Transform the cartesian points to fractional coordinates
    transformed_points = transform_coordinates(transformation_matrix_bohr, coords_bohr)

    # Create a dataframe with the ionic charges too, corresponsing to the correct elements.
    added_transformed_points = []
    t=0
    for element in df['El']:
        if element == 'Si':
            added_transformed_points.append(np.append(transformed_points[t], +4))
        else:
            added_transformed_points.append(np.append(transformed_points[t], -4))
        t=t+1
    added_transformed_points = np.array(added_transformed_points)
    df_fractional_coords_transformed = pd.DataFrame(added_transformed_points)

    return df_fractional_coords_transformed, df_cartesian_coords_bohr


# Process 3x3x1 with C centre (defect placed at Carbon site)

In [13]:
df_3x3x1 = pd.read_csv('Cartesian_coords_armstrong/3x3x1_Cr_cartesian_coords_C_centre.csv', delim_whitespace=True)
v_3x3x1 = [(4.62,-8.00207473,0.0), (4.62,8.00207473,0.0), (0.0,0.0,10.07000000)]

print(fractional_coordinates(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2], df_3x3x1))
print(params_and_units(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2]))

(           0         1         2    3
0   0.333333  0.000000 -0.312106  4.0
1   0.333333  0.000000  0.187894  4.0
2   0.333333  0.000000 -0.499540 -4.0
3   0.333333  0.000000  0.000460 -4.0
4   0.222222 -0.222222 -0.062158  4.0
..       ...       ...       ...  ...
67  0.000000 -0.333333  0.000460 -4.0
68 -0.111111 -0.555556 -0.062158  4.0
69  0.111111 -0.111111  0.437842  4.0
70 -0.111111 -0.555556 -0.250197 -4.0
71  0.111111 -0.111111  0.249803 -4.0

[72 rows x 4 columns],            0         1         2
0   5.820356  0.000000 -5.939234
1   5.820356  0.000000  3.575537
2   5.820356  0.000000 -9.506017
3   5.820356  0.000000  0.008754
4   5.820356 -3.360384 -1.182838
..       ...       ...       ...
67  2.910178 -5.040576  0.008754
68  2.910178 -8.400960 -1.182838
69  2.910178 -1.680192  8.331932
70  2.910178 -8.400960 -4.761134
71  2.910178 -1.680192  4.753636

[72 rows x 3 columns])
(17.46106813307947, 17.46106813307947, 19.029540705202, 90.0, 90.0, 119.99999999399628)


In [ ]:
fractional_coordinates(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2], df_3x3x1)[1].to_csv('BOHR_3x3x1_Cr_carterian_coords_C_centre.csv', sep='\t', index=False)
fractional_coordinates(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2], df_3x3x1)[0].to_csv('BOHR_3x3x1_Cr_fractional_coords_C_centre.csv', sep='\t', index=False)

# Process 3x3x1 with Si centre (defect placed at Silicon site)

In [14]:
df_3x3x1 = pd.read_csv('Cartesian_coords_armstrong/3x3x1_Cr_cartesian_coords_Si_centre.csv', delim_whitespace=True)
v_3x3x1 = [(4.62,-8.00207473,0.0), (4.62,8.00207473,0.0), (0.0,0.0,10.07000000)]

print(fractional_coordinates(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2], df_3x3x1))
print(params_and_units(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2]))

(           0         1         2    3
0   0.333333  0.000000 -0.312106 -4.0
1   0.333333  0.000000  0.187894 -4.0
2   0.333333  0.000000 -0.499540  4.0
3   0.333333  0.000000  0.000460  4.0
4   0.222222 -0.222222 -0.062158 -4.0
..       ...       ...       ...  ...
67  0.000000 -0.333333  0.000460  4.0
68 -0.111111 -0.555556 -0.062158 -4.0
69  0.111111 -0.111111  0.437842 -4.0
70 -0.111111 -0.555556 -0.250197  4.0
71  0.111111 -0.111111  0.249803  4.0

[72 rows x 4 columns],            0         1         2
0   5.820356  0.000000 -5.939234
1   5.820356  0.000000  3.575537
2   5.820356  0.000000 -9.506017
3   5.820356  0.000000  0.008754
4   5.820356 -3.360384 -1.182838
..       ...       ...       ...
67  2.910178 -5.040576  0.008754
68  2.910178 -8.400960 -1.182838
69  2.910178 -1.680192  8.331932
70  2.910178 -8.400960 -4.761134
71  2.910178 -1.680192  4.753636

[72 rows x 3 columns])
(17.46106813307947, 17.46106813307947, 19.029540705202, 90.0, 90.0, 119.99999999399628)


In [15]:
fractional_coordinates(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2], df_3x3x1)[1].to_csv('BOHR_3x3x1_Cr_carterian_coords_Si_centre.csv', sep='\t', index=False)
fractional_coordinates(v_3x3x1[0], v_3x3x1[1], v_3x3x1[2], df_3x3x1)[0].to_csv('BOHR_3x3x1_Cr_fractional_coords_Si_centre.csv', sep='\t', index=False)

# Process 5x5x2

In [11]:
df_5x5x2 = pd.read_csv('Cartesian_coords_armstrong/5x5x2_Cr_cartesian_coords.csv', delim_whitespace=True)
v_5x5x2 = [(7.7, -13.33679122, 0.0),(7.7, 13.33679122, 0.0),(0.0, 0.0, 20.14)]

print(fractional_coordinates(v_5x5x2[0], v_5x5x2[1], v_5x5x2[2], df_5x5x2))
print(params_and_units(v_5x5x2[0], v_5x5x2[1], v_5x5x2[2]))

(            0             1         2    3
0    0.400000 -1.331922e-16  0.093947 -4.0
1    0.400000 -1.331922e-16  0.343947 -4.0
2    0.400000 -1.331922e-16  0.000230  4.0
3    0.400000 -1.331922e-16  0.250230  4.0
4    0.333333 -1.333333e-01  0.218921 -4.0
..        ...           ...       ...  ...
395  0.200000 -2.000000e-01 -0.249770  4.0
396  0.133333 -3.333333e-01 -0.281079 -4.0
397  0.266667 -6.666667e-02 -0.031079 -4.0
398  0.133333 -3.333333e-01 -0.375098  4.0
399  0.266667 -6.666667e-02 -0.125099  4.0

[400 rows x 4 columns],              0             1          2
0    11.640712 -3.356828e-15   3.575537
1    11.640712 -3.356828e-15  13.090307
2    11.640712 -3.356828e-15   0.008754
3    11.640712 -3.356828e-15   9.523524
4    11.640712 -3.360384e+00   8.331932
..         ...           ...        ...
395   8.730534 -5.040576e+00  -9.506017
396   8.730534 -8.400960e+00 -10.697609
397   8.730534 -1.680192e+00  -1.182838
398   8.730534 -8.400960e+00 -14.275904
399   8.730534 -1.

In [7]:
fractional_coordinates(v_5x5x2[0], v_5x5x2[1], v_5x5x2[2], df_5x5x2)[1].to_csv('BOHR_5x5x2_Cr_carterian_coords.csv', sep='\t', index=False)

In [8]:
fractional_coordinates(v_5x5x2[0], v_5x5x2[1], v_5x5x2[2], df_5x5x2)[0].to_csv('BOHR_5x5x2_Cr_fractional_coords.csv', sep='\t', index=False)

# Process 3x3x2

In [10]:
df_3x3x2 = pd.read_csv('Cartesian_coords_armstrong/3x3x2_Cr_cartesian_coords.csv', delim_whitespace=True)
v_3x3x2 = [(4.62, -8.002074730968213,0.0),(4.62, 8.002074730968213, 0.0),(0.0, 0.0, 20.14)]

print(fractional_coordinates(v_3x3x2[0], v_3x3x2[1], v_3x3x2[2], df_3x3x2))
print(params_and_units(v_3x3x2[0], v_3x3x2[1], v_3x3x2[2]))

(            0         1         2    3
0    0.333333  0.000000  0.093947 -4.0
1    0.333333  0.000000  0.343947 -4.0
2    0.333333  0.000000  0.000230  4.0
3    0.333333  0.000000  0.250230  4.0
4    0.222222 -0.222222  0.218921 -4.0
..        ...       ...       ...  ...
139  0.333333  0.000000 -0.249770  4.0
140  0.222222 -0.222222 -0.281079 -4.0
141  0.444444  0.222222 -0.031079 -4.0
142  0.222222 -0.222222 -0.375098  4.0
143  0.444444  0.222222 -0.125099  4.0

[144 rows x 4 columns],             0         1          2
0    5.820356  0.000000   3.575537
1    5.820356  0.000000  13.090307
2    5.820356  0.000000   0.008754
3    5.820356  0.000000   9.523524
4    5.820356 -3.360384   8.331932
..        ...       ...        ...
139  5.820356  0.000000  -9.506017
140  5.820356 -3.360384 -10.697609
141  5.820356  3.360384  -1.182838
142  5.820356 -3.360384 -14.275904
143  5.820356  3.360384  -4.761134

[144 rows x 3 columns])
(17.461068134664, 17.461068134664, 38.059081410404, 90.0, 90.

In [12]:
fractional_coordinates(v_3x3x2[0], v_3x3x2[1], v_3x3x2[2], df_3x3x2)[1].to_csv('BOHR_3x3x2_Cr_carterian_coords.csv', sep='\t', index=False)
fractional_coordinates(v_3x3x2[0], v_3x3x2[1], v_3x3x2[2], df_3x3x2)[0].to_csv('BOHR_3x3x2_Cr_fractional_coords.csv', sep='\t', index=False)